# 웹 스크래핑 프로그램 만들어보자

1. 예제 페이지에서 여행기사를 추출한다.
   https://korean.visitkorea.or.kr/main/main.do
2. 검색어와 스크래핑 건수를 입력받는다.
3. 페이지를 이동한다.
4. 상세 페이지별로 여행기사 정보를 수집한다.
5.  이미지와 텍스트를 추출해서 파일로 저장까지

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import math, time

options = Options()
options.add_argument('--window-size=974,1047')
options.add_argument('--window-position=-7,0')
options.add_experimental_option("detach", True)

In [3]:
search = input('검색어:')
cnt = int(input('크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수 

검색어: 제주도
크롤링 할 건수는 몇건입니까?:  290


In [4]:
URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword='+search
driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(3)

In [5]:
# 여행기사 더보기 클릭
# driver.find_element(By.CSS_SELECTOR, ".more_view").click() 중복주의
driver.find_element(By.CSS_SELECTOR, "#s_recommend > .more_view > a").click()

In [6]:
# 제목 추출
# 왜 길이가 34일까?
result = driver.find_elements(By.CSS_SELECTOR,'.tit a')
len(result)

34

In [7]:
for i in result:
    print(i.text)

























해변산책부터 레이싱까지, 제주 반려동물 동반여행 추천 코스
제주도가 품은 신비의 화산 숲, 환상숲곶자왈공원
한국관광공사 추천 2월 걷기여행길, 이른 봄을 맞이하는 제주도 걷기길
제주도의 숨겨진 재미를 찾는
<놀멍, 배우멍 제주마을체험 - 1일형 체험학습>
제주도 조천읍 버킷리스트 여행지 작성하기
김창열의 회귀 철학을 건축으로 표현한 미술관, 제주도립김창열미술관
[국내 트레킹 추천]
제주 올레길 걷기 여행,
준비부터 코스 선택까지 꿀팁 총정리!
4월 제주도 가볼 만한, 가파도 청보리 축제
제주도 유채꽃 명소 추천, 봄에 떠나는 인생샷 여행
소소한 '제주여행' 꿀팁! 〈겨울편〉


In [11]:
# 더 정확하게 찾아주자
result = driver.find_elements(By.CSS_SELECTOR,'#search_result .tit>a')
len(result)

10

In [17]:
# 한페이지내 콘텐츠별 상세페이지 정보를 추출해보자
# 페이지 로딩시간 고려해서 넉넉히 2~3초 간격을 주자
from bs4 import BeautifulSoup
contents_no = 0

for item in result:
    contents_no += 1
    item.click()
    time.sleep(3)  
    
    print(f'======= [ {contents_no} ]  =======')
    html = driver.page_source

    soup = BeautifulSoup(html, 'lxml')
    title = soup.find(id='topTitle')
    print(title.text)

    driver.back()
    time.sleep(3)

print('==== 완료 ====')

ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:goSearchDetail('5bf42aa9-c0dd-4456-a34b-478f953766df');">...</a> is not clickable at point (191, 9). Other element would receive the click: <h1 class="logo">...</h1>
  (Session info: chrome=126.0.6478.126)
Stacktrace:
	GetHandleVerifier [0x00007FF71175EEA2+31554]
	(No symbol) [0x00007FF7116D7ED9]
	(No symbol) [0x00007FF71159872A]
	(No symbol) [0x00007FF7115F012E]
	(No symbol) [0x00007FF7115EDAF2]
	(No symbol) [0x00007FF7115EAF8B]
	(No symbol) [0x00007FF7115EA156]
	(No symbol) [0x00007FF7115DC151]
	(No symbol) [0x00007FF71160D02A]
	(No symbol) [0x00007FF7115DBA76]
	(No symbol) [0x00007FF71160D240]
	(No symbol) [0x00007FF71162C977]
	(No symbol) [0x00007FF71160CDD3]
	(No symbol) [0x00007FF7115DA33B]
	(No symbol) [0x00007FF7115DAED1]
	GetHandleVerifier [0x00007FF711A68B1D+3217341]
	GetHandleVerifier [0x00007FF711AB5AE3+3532675]
	GetHandleVerifier [0x00007FF711AAB0E0+3489152]
	GetHandleVerifier

In [18]:
result = driver.find_elements(By.CSS_SELECTOR,'#search_result .tit>a')
result[0].text

'해변산책부터 레이싱까지, 제주 반려동물 동반여행 추천 코스'

In [19]:
# result[0].click() ## 에러 잘남
from selenium.webdriver.common.keys import Keys
item.send_keys(Keys.ENTER)

In [20]:
from selenium.webdriver.common.keys import Keys

def page_work():
    result = driver.find_elements(By.CSS_SELECTOR,'#search_result .tit>a')
    global contents_no, cnt
    
    for item in result:
        contents_no += 1
        
        if contents_no <= cnt :    
            print(f'[콘텐츠 {contents_no}]')  
            item.send_keys(Keys.ENTER) # .click()은 에러 잘남

            time.sleep(2)
            html = driver.page_source
            html_dom = BeautifulSoup(html, 'lxml')

            title = html_dom.find(id='topTitle')
            print(title.text)
            driver.back()
            time.sleep(2)     

## 페이지이동 스크래핑

In [22]:
contents_no = 0
print('스크래핑 프로그램 실행')

for page_no in range(1, page_cnt+1):    
    print(f'====== {page_no} 페이지 스크래핑 시작 ======')
    page_work()
    print(f'====== {page_no} 페이지 스크래핑 완료 ======')
    if page_no < page_cnt:
        driver.find_element(By.XPATH, f'/html/body/div[3]/div/div[1]/div[14]/a[{page_no+1}]').click()
        time.sleep(2)
print('스크래핑 프로그램 종료')

driver.close()

스크래핑 프로그램 실행
====== 1 페이지 스크래핑 시작 ======
====== 1 페이지 스크래핑 완료 ======


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div[1]/div[14]/a[2]"}
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71175EEA2+31554]
	(No symbol) [0x00007FF7116D7ED9]
	(No symbol) [0x00007FF71159872A]
	(No symbol) [0x00007FF7115E8434]
	(No symbol) [0x00007FF7115E853C]
	(No symbol) [0x00007FF71162F6A7]
	(No symbol) [0x00007FF71160D06F]
	(No symbol) [0x00007FF71162C977]
	(No symbol) [0x00007FF71160CDD3]
	(No symbol) [0x00007FF7115DA33B]
	(No symbol) [0x00007FF7115DAED1]
	GetHandleVerifier [0x00007FF711A68B1D+3217341]
	GetHandleVerifier [0x00007FF711AB5AE3+3532675]
	GetHandleVerifier [0x00007FF711AAB0E0+3489152]
	GetHandleVerifier [0x00007FF71180E776+750614]
	(No symbol) [0x00007FF7116E375F]
	(No symbol) [0x00007FF7116DEB14]
	(No sy

## 상세페이지 콘텐츠도 함께 추출해서 파일로 저장

In [23]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import math, time, pandas as pd, urllib.request

options = Options()
options.add_argument('--window-size=974,1047')
options.add_argument('--window-position=-7,0')
options.add_experimental_option("detach", True)

In [24]:
search = input('검색어:')
cnt = int(input('스크래핑 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수 

검색어: 제주도
스크래핑 할 건수는 몇건입니까?:  290


In [25]:
URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword='+search
driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(2)

In [26]:
# 여행기사 더보기 클릭
# driver.find_element(By.CSS_SELECTOR, ".more_view").click() 중복주의
driver.find_element(By.CSS_SELECTOR, "#s_recommend > .more_view > a").click()

In [27]:
title_list = []
contents_list = []

def page_work():
    result = driver.find_elements(By.CSS_SELECTOR,'#search_result .tit>a')
    global contents_no, cnt
    global title_list, contents_list
    
    for item in result:
        contents_no += 1
        
        if contents_no <= cnt :    
            print(f'[콘텐츠 {contents_no}]')  
            item.send_keys(Keys.ENTER) # .click()은 에러 잘남

            time.sleep(2)
            html = driver.page_source
            html_dom = BeautifulSoup(html, 'lxml')

            title = html_dom.find(id='topTitle')
            title_list.append(title.text)
            print(title.text)
            
            contents = driver.find_elements(By.CLASS_NAME, 'txt_p')
            contents_merge = ' '.join([item.text for item in contents])        
            contents_list.append(contents_merge)
            
            driver.back()
            time.sleep(2)     

In [29]:
contents_no = 0
today = time.localtime()
print('스크래핑 프로그램 실행')

for page_no in range(1, page_cnt+1):    
    print(f'====== {page_no} 페이지 스크래핑 시작 ======')
    page_work()
    print(f'====== {page_no} 페이지 스크래핑 완료 ======')
    
    DF = pd.DataFrame({"제목":title_list, "내용":contents_list})
    filename = f'{search}여행기사_{cnt}건_{today.tm_year}{today.tm_mon}{today.tm_mday}.xlsx'
    DF.to_excel(filename)
    print(f'====== {page_no} 페이지 {filename}파일 저장 완료 ======')
    
    if page_no < page_cnt:
        driver.find_element(By.XPATH, f'/html/body/div[3]/div/div[1]/div[14]/a[{page_no+1}]').click()
        time.sleep(2)
                   
print('스크래핑 프로그램 종료')
driver.close()

스크래핑 프로그램 실행
====== 1 페이지 스크래핑 시작 ======
====== 1 페이지 스크래핑 완료 ======
====== 1 페이지 제주도여행기사_290건_2024626.xlsx파일 저장 완료 ======


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div[1]/div[14]/a[2]"}
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71175EEA2+31554]
	(No symbol) [0x00007FF7116D7ED9]
	(No symbol) [0x00007FF71159872A]
	(No symbol) [0x00007FF7115E8434]
	(No symbol) [0x00007FF7115E853C]
	(No symbol) [0x00007FF71162F6A7]
	(No symbol) [0x00007FF71160D06F]
	(No symbol) [0x00007FF71162C977]
	(No symbol) [0x00007FF71160CDD3]
	(No symbol) [0x00007FF7115DA33B]
	(No symbol) [0x00007FF7115DAED1]
	GetHandleVerifier [0x00007FF711A68B1D+3217341]
	GetHandleVerifier [0x00007FF711AB5AE3+3532675]
	GetHandleVerifier [0x00007FF711AAB0E0+3489152]
	GetHandleVerifier [0x00007FF71180E776+750614]
	(No symbol) [0x00007FF7116E375F]
	(No symbol) [0x00007FF7116DEB14]
	(No sy

In [ ]:
## 이미지 추출 기능 추가

In [30]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import math, time, os, pandas as pd

options = Options()
options.add_argument('--window-size=974,1047')
options.add_argument('--window-position=-7,0')
options.add_experimental_option("detach", True)

In [31]:
search = input('검색어:')
cnt = int(input('스크래핑 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수 

검색어: 제주도
스크래핑 할 건수는 몇건입니까?:  290


In [32]:
now = time.localtime()
date_format = '%04d%02d%02d'%(now.tm_year, now.tm_mon, now.tm_mday)
f_dir = f'{os.getcwd()}\\{search}여행기사_{cnt}건_{date_format}'
os.makedirs(f_dir)

In [33]:
URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword='+search
driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(2)
# 여행기사 더보기 클릭
driver.find_element(By.CSS_SELECTOR, "#s_recommend > .more_view > a").click()
time.sleep(2)

In [34]:
title_list = []
contents_list = []
img_url_list = []

def page_work():
    result = driver.find_elements(By.CSS_SELECTOR,'#search_result .tit>a')
    global contents_no, cnt
    global title_list, contents_list, img_url_list
    
    for item in result:
        contents_no += 1
        
        if contents_no <= cnt :    
            print(f'[콘텐츠 {contents_no}]')  
            item.send_keys(Keys.ENTER) # .click()은 에러 잘남

            time.sleep(2)
            
            # 이미지 추출을 위해 미리 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            
            html = driver.page_source
            html_dom = BeautifulSoup(html, 'lxml')

            title = html_dom.find(id='topTitle')
            title_list.append(title.text)
            print(title.text)
            
            img_tag_list = html_dom.select('.img_typeBox img')
            img_url_list = [item['src'] for item in img_tag_list]

            contents = driver.find_elements(By.CLASS_NAME, 'txt_p')
            contents_merge = ' '.join([item.text for item in contents])        
            contents_list.append(contents_merge)           
            
            driver.back()
            time.sleep(2)     

In [35]:
def file_export():
    
    DF = pd.DataFrame({"제목":title_list, "내용":contents_list})
    filename = f'{search}여행기사_{cnt}건_{date_format}.xlsx'
    DF.to_excel(f_dir+'\\'+filename)
    print(f'====== {page_no} 페이지 {filename} 파일 저장 완료 ======')
    
    
    no = 0
    for src in img_url_list:
        # 다운로드  (주소, 파일이름)
        urllib.request.urlretrieve(src, f'{f_dir}\\{page_no}_{no}.jpg')
        no += 1
    print(f'====== {page_no} 페이지 {f_dir} 디렉토리 이미지 저장 완료 ======')

In [36]:
contents_no = 0
today = time.localtime()
print('스크래핑 프로그램 실행')

for page_no in range(1, page_cnt+1):    
    print(f'====== {page_no} 페이지 스크래핑 시작 ======')
    page_work()
    print(f'====== {page_no} 페이지 스크래핑 작업중 ======')
    file_export()
    print(f'====== {page_no} 페이지 스크래핑 완료 ======')
    if page_no < page_cnt:
        driver.find_element(By.XPATH, f'/html/body/div[3]/div/div[1]/div[14]/a[{page_no+1}]').click()
        time.sleep(2)
                   
print('스크래핑 프로그램 종료')
driver.close()

스크래핑 프로그램 실행
====== 1 페이지 스크래핑 시작 ======
[콘텐츠 1]
해변산책부터 레이싱까지, 제주 반려동물 동반여행 추천 코스
[콘텐츠 2]


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71175EEA2+31554]
	(No symbol) [0x00007FF7116D7ED9]
	(No symbol) [0x00007FF71159872A]
	(No symbol) [0x00007FF7115AADBD]
	(No symbol) [0x00007FF7115AB84A]
	(No symbol) [0x00007FF71159F5BB]
	(No symbol) [0x00007FF71159D326]
	(No symbol) [0x00007FF7115A0EB1]
	(No symbol) [0x00007FF7115A0F50]
	(No symbol) [0x00007FF7115E9192]
	(No symbol) [0x00007FF7115E9272]
	(No symbol) [0x00007FF7115DF119]
	(No symbol) [0x00007FF71160D02A]
	(No symbol) [0x00007FF7115DBA76]
	(No symbol) [0x00007FF71160D240]
	(No symbol) [0x00007FF71162C977]
	(No symbol) [0x00007FF71160CDD3]
	(No symbol) [0x00007FF7115DA33B]
	(No symbol) [0x00007FF7115DAED1]
	GetHandleVerifier [0x00007FF711A68B1D+3217341

In [39]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import math, time, os, pandas as pd, urllib.request

options = Options()
options.add_argument('--window-size=974,1047')
options.add_argument('--window-position=-7,0')
options.add_experimental_option("detach", True)

search = input('검색어:')
cnt = int(input('스크래핑 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수 

now = time.localtime()
date_format = '%04d%02d%02d'%(now.tm_year, now.tm_mon, now.tm_mday)
f_dir = f'{os.getcwd()}\\{search}여행기사_{cnt}건_{date_format}'
os.makedirs(f_dir)

URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword='+search
driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(2)
# 여행기사 더보기 클릭
driver.find_element(By.CSS_SELECTOR, "#s_recommend > .more_view > a").click()
time.sleep(2)

title_list = []
contents_list = []
img_url_list = []

def page_work():
    result = driver.find_elements(By.CSS_SELECTOR,'#search_result .tit>a')
    global contents_no, cnt
    global title_list, contents_list, img_url_list

    for item in result:
        contents_no += 1

        if contents_no <= cnt :    
            print(f'[콘텐츠 {contents_no}]')  
            item.send_keys(Keys.ENTER) # .click()은 에러 잘남

            time.sleep(2)

            # 이미지 추출을 위해 미리 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

            html = driver.page_source
            html_dom = BeautifulSoup(html, 'lxml')

            title = html_dom.find(id='topTitle')
            title_list.append(title.text)
            print(title.text)

            img_tag_list = html_dom.select('.img_typeBox img')
            img_url_list = [item['src'] for item in img_tag_list]

            contents = driver.find_elements(By.CLASS_NAME, 'txt_p')
            contents_merge = ' '.join([item.text for item in contents])        
            contents_list.append(contents_merge)           

            driver.back()
            time.sleep(2)     


def file_export():

    DF = pd.DataFrame({"제목":title_list, "내용":contents_list})
    filename = f'{search}여행기사_{cnt}건_{date_format}.xlsx'
    DF.to_excel(f_dir+'\\'+filename)
    print(f'====== {page_no} 페이지 {filename} 파일 저장 완료 ======')


    no = 0
    for src in img_url_list:
        # 다운로드  (주소, 파일이름)
        urllib.request.urlretrieve(src, f'{f_dir}\\{page_no}_{no}.jpg')
        no += 1
    print(f'====== {page_no} 페이지 {f_dir} 디렉토리 이미지 저장 완료 ======')


contents_no = 0
today = time.localtime()
print('스크래핑 프로그램 실행')

for page_no in range(1, page_cnt+1):    
    print(f'====== {page_no} 페이지 스크래핑 시작 ======')
    page_work()
    print(f'====== {page_no} 페이지 스크래핑 작업중 ======')
    file_export()
    print(f'====== {page_no} 페이지 스크래핑 완료 ======')
    if page_no < page_cnt:
        driver.find_element(By.XPATH, f'/html/body/div[3]/div/div[1]/div[14]/a[{page_no+1}]').click()
        time.sleep(2)

print('스크래핑 프로그램 종료')
driver.close()

검색어: 제주도
스크래핑 할 건수는 몇건입니까?:  20


스크래핑 프로그램 실행
====== 1 페이지 스크래핑 시작 ======
[콘텐츠 1]
해변산책부터 레이싱까지, 제주 반려동물 동반여행 추천 코스
[콘텐츠 2]


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71175EEA2+31554]
	(No symbol) [0x00007FF7116D7ED9]
	(No symbol) [0x00007FF71159872A]
	(No symbol) [0x00007FF7115AADBD]
	(No symbol) [0x00007FF7115AB84A]
	(No symbol) [0x00007FF71159F5BB]
	(No symbol) [0x00007FF71159D326]
	(No symbol) [0x00007FF7115A0EB1]
	(No symbol) [0x00007FF7115A0F50]
	(No symbol) [0x00007FF7115E9192]
	(No symbol) [0x00007FF7115E9272]
	(No symbol) [0x00007FF7115DF119]
	(No symbol) [0x00007FF71160D02A]
	(No symbol) [0x00007FF7115DBA76]
	(No symbol) [0x00007FF71160D240]
	(No symbol) [0x00007FF71162C977]
	(No symbol) [0x00007FF71160CDD3]
	(No symbol) [0x00007FF7115DA33B]
	(No symbol) [0x00007FF7115DAED1]
	GetHandleVerifier [0x00007FF711A68B1D+3217341

In [37]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import math, time, os, pandas as pd, urllib.request

options = Options()
options.add_argument('--window-size=974,1047')
options.add_argument('--window-position=-7,0')
options.add_experimental_option("detach", True)

search = input('검색어:')
cnt = int(input('스크래핑 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수 

now = time.localtime()
date_format = '%04d%02d%02d' % (now.tm_year, now.tm_mon, now.tm_mday)
f_dir = os.path.join(os.getcwd(), f'{search}여행기사_{cnt}건_{date_format}')
os.makedirs(f_dir, exist_ok=True)

URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword=' + search
driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(2)

# 여행기사 더보기 클릭
driver.find_element(By.CSS_SELECTOR, "#s_recommend > .more_view > a").click()
time.sleep(2)

title_list = []
contents_list = []
img_url_list = []

def page_work():
    global contents_no, cnt
    global title_list, contents_list, img_url_list

    result = driver.find_elements(By.CSS_SELECTOR, '#search_result .tit>a')

    for item in result:
        contents_no += 1
        if contents_no > cnt:
            return

        print(f'[콘텐츠 {contents_no}]')
        item.send_keys(Keys.ENTER)  # .click()은 에러 잘남
        time.sleep(2)

        # 이미지 추출을 위해 미리 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        html = driver.page_source
        html_dom = BeautifulSoup(html, 'lxml')

        title = html_dom.select_one('#topTitle')
        if title:
            title_list.append(title.text)
            print(title.text)
        else:
            title_list.append('No Title')

        img_tag_list = html_dom.select('.img_typeBox img')
        img_url_list.extend([item['src'] for item in img_tag_list])

        contents = driver.find_elements(By.CLASS_NAME, 'txt_p')
        contents_merge = ' '.join([item.text for item in contents])
        contents_list.append(contents_merge)

        driver.back()
        time.sleep(2)

def file_export(page_no):
    DF = pd.DataFrame({"제목": title_list, "내용": contents_list})
    filename = f'{search}여행기사_{cnt}건_{date_format}.xlsx'
    DF.to_excel(os.path.join(f_dir, filename))
    print(f'====== {page_no} 페이지 {filename} 파일 저장 완료 ======')

    for i, src in enumerate(img_url_list):
        try:
            # 다운로드 (주소, 파일이름)
            urllib.request.urlretrieve(src, os.path.join(f_dir, f'{page_no}_{i}.jpg'))
        except Exception as e:
            print(f'이미지 다운로드 실패 {src}: {e}')

    print(f'====== {page_no} 페이지 {f_dir} 디렉토리 이미지 저장 완료 ======')

contents_no = 0
print('스크래핑 프로그램 실행')

for page_no in range(1, page_cnt + 1):
    print(f'====== {page_no} 페이지 스크래핑 시작 ======')
    page_work()
    print(f'====== {page_no} 페이지 스크래핑 작업 중 ======')
    file_export(page_no)
    print(f'====== {page_no} 페이지 스크래핑 완료 ======')
    if page_no < page_cnt:
        try:
            next_button = driver.find_element(By.XPATH, f'/html/body/div[3]/div/div[1]/div[14]/a[{page_no + 1}]')
            next_button.click()
            time.sleep(2)
        except Exception as e:
            print(f'페이지 넘김 실패: {e}')
            break

print('스크래핑 프로그램 종료')
driver.close()


검색어: 제주도
스크래핑 할 건수는 몇건입니까?:  20


스크래핑 프로그램 실행
====== 1 페이지 스크래핑 시작 ======
[콘텐츠 1]
해변산책부터 레이싱까지, 제주 반려동물 동반여행 추천 코스
[콘텐츠 2]


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71175EEA2+31554]
	(No symbol) [0x00007FF7116D7ED9]
	(No symbol) [0x00007FF71159872A]
	(No symbol) [0x00007FF7115AADBD]
	(No symbol) [0x00007FF7115AB84A]
	(No symbol) [0x00007FF71159F5BB]
	(No symbol) [0x00007FF71159D326]
	(No symbol) [0x00007FF7115A0EB1]
	(No symbol) [0x00007FF7115A0F50]
	(No symbol) [0x00007FF7115E9192]
	(No symbol) [0x00007FF7115E9272]
	(No symbol) [0x00007FF7115DF119]
	(No symbol) [0x00007FF71160D02A]
	(No symbol) [0x00007FF7115DBA76]
	(No symbol) [0x00007FF71160D240]
	(No symbol) [0x00007FF71162C977]
	(No symbol) [0x00007FF71160CDD3]
	(No symbol) [0x00007FF7115DA33B]
	(No symbol) [0x00007FF7115DAED1]
	GetHandleVerifier [0x00007FF711A68B1D+3217341